---

#University of Stirling - Spring 2023

## CSCU9M6 - Natural Language Processing and Computer Vision (2022/3)

---

# Assignment Summary

In this activity, you are required to apply the knowledge acquired in this module through the design and development of a complete project for image classification in an application to be defined by yourself. For this, you will need to perform the following **mandatory** steps:

1. [Problem definition](#scrollTo=hglJVRRslqMn)
2. [GitHub repository](#scrollTo=ecxDhkV9qmUf)
3. [Dataset](#scrollTo=qEgFzxmWrGA9)
4. [Dataloader](#scrollTo=EDd6lLwlx4un)
5. [Proposed solution](#scrollTo=ScTrpUW8zOp4)
6. [Experimental tests and evaluations](#scrollTo=3RBW58of0ZDo)
7. [Quiz and Report](#scrollTo=ws14iV4Dp_vf)

**Deadlines** and other details can be seen on Canvas [\[link\]](https://canvas.stir.ac.uk/courses/12587/assignments/102373).

---

# 1. **Problem definition** 


In this assignment, you are required to apply the knowledge acquired in the module to solve a classification problem from images collected in the context of two different cities (A and B).
 - If the work is being carried out in pairs, **cities A and B must be the hometowns of each student**. In the case of individual work, city A must be your hometown and city B must be Stirling (or Edinburgh, if needed).
 - The standard recommendation is that the project focuses on classifying cars or trees image scenes, which are easier to identify and annotate. Other objects or phenomena can be adopted, but are subject to prior approval by the module instructor (Jefersson A. dos Santos). **You are not allowed to assemble datasets containing people. Other sensitive patterns, such as license plates, must be properly hidden.**
 - Don't panic! We are aware that acquiring images _in situ_ is an impediment for most students. The dataset can be assembled with images collected remotely or from public repositories. Just be careful with rights and permissions for using images found on the internet. Anyway, these factors must be taken into account for the problem definition.
 - While we encourage you to do interesting and engaging work, it shouldn't be too complex or time-consuming. Try to appropriately scale the time required for this step. Ask the instructors for advice, if necessary. **GA students:** you are encouraged to link the project with your work activities, but keep in mind you still need to construct two datasets (A and B). 

[top](#scrollTo=4i5afvUbhmGo)
 

---
# 2. **GitHub repository**

Give your project a name, create a private [GitHub repository](https://github.com/) with the name [Module Code] + [Project Name] and give access to the module instructors. Create a cover page with a description of your project. This empty notebook must be uploaded in the repository as well as the created dataset. The deadline to perform this task will be 10 days after the publication of this notebook. 
This notebook should be updated and committed to the repository according to the deadlines.
The repository's update history will be used as a criterion for monitoring and evaluating the work.
**Check the videos provided in the extra section on Canvas for more details on how to create your GitHub repository** [\[link\]](https://canvas.stir.ac.uk/courses/12587/pages/extra-session-cnn-hyperparameters-and-github).

[top](#scrollTo=4i5afvUbhmGo)

---
# 3. **Dataset creation**

You must collect a minimum of **200 positive samples** from the study objects for each city (A and B). 
Note that, depending on the task being solved, it will also be necessary to collect more samples - negative ones, for instance.

Your dataset can be assembled from one or more of the following ways:

  - *M1* - Pictures taken by yourself on site (street view from cities A and B), with attention to anonymization issues (if it is the case). It is not allowed to assemble datasets containing people. Other sensitive patterns, such as license plates, must be properly hidden.

  - *M2* - Aerial satellite/drone images obtained from GIS and remote sensing platforms or public repositories. Be careful with unusual file formats that may be challenging to manipulate using basic image processing libraries. We recommend keeping or converting the images to jpg or png.

  - *M3* - Pictures taken from other public available datasets. Remember you are not allowed to use datasets containing people or other sensitive patterns/objects.

  - *M4* - Images crawled from the internet as a whole (social networks, webpages, etc), with special attention to use and copyrights.

  - *M5* - Textual and metadata you may need in your project, with special attention to use and copyrights (as always!).

**Important:** If you collect the images on your own or from aerial imagery repositories, it will be necessary to keep the geographic coordinates. If you collect from specific websites, please retain the source links. This information should be placed in a .csv file and made available along with the final dataset.

[top](#scrollTo=4i5afvUbhmGo)

---

# 4. **Dataloader**

Here you are required to implement all the code related to pre-processing, cleaning, de-noising and preparing the input images and metadata according to the necessary data structures as input to your pattern recognition module. We recommend using [PyTorch](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) or [Tensorflow (with Keras)](https://keras.io/getting_started/intro_to_keras_for_engineers/) as a base, but you are free to use any library or platform as long as it is well justified in the [final report](#scrollTo=ws14iV4Dp_vf).

[top](scrollTo=4i5afvUbhmGo)

In [1]:
# import necessary libraries

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16

import keras
import keras.utils
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import time

In [4]:
# Set image size
img_size = (224, 224)

# Set batch size
batch_size = 32

# Define image data generator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Define the paths to the image files and the labels
image_dir = 'C:/Users/maria/Documents/GitHub/CSCU9M6-2929300/Dataset/train'
label_file = 'C:/Users/maria/Documents/GitHub/CSCU9M6-2929300/'


# Define the labels
labels = ['ara', 'ban', 'che', 'pal', 'pla', 'art', 'ced', 
          'ash', 'syc', 'oak', 'app', 'pin', 'bee', 'bct', 'not']

def load_and_resize(filename, label):
    # Load the image
    image = tf.io.read_file(filename)
    # Decode the image
    image = tf.image.decode_jpeg(image, channels=3)
    # Resize the image
    image = tf.image.resize(image, [img_size[0], img_size[1]])
    # Rescale the pixel values
    image = image / 255.0
    # Return the image and label
    return image, label

def create_dataset(filenames, labels, batch_size):
    # Create TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    # Shuffle the dataset
    dataset = dataset.shuffle(buffer_size=len(filenames), reshuffle_each_iteration=False)
    # Load and preprocess the images and labels in parallel
    dataset = dataset.map(
        map_func=load_and_resize,
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # Batch the dataset
    dataset = dataset.batch(batch_size)
    # Prefetch the dataset
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    # Return the dataset
    return dataset

# Create a function to load and preprocess the image and its label
def load_and_preprocess_image(filename, label):
    # Load the image
    image_string = tf.io.read_file(filename)
    # Decode the image
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    # Preprocess the image
    image, label = preprocess_image(image_decoded, label)
    # Return the image and its label
    return image, label

# Load the labels from the file
# Load the labels from the file
with open(os.path.join(label_file, 'imageData.txt')) as f:
    lines = f.readlines()
    label_dict = {}
    for line in lines:
        filename, label, _ = line.strip().split('.', maxsplit=2) # added maxsplit to prevent an error i was
        if label in labels:                                      # getting due to the way I named my files
               label_dict[os.path.join(image_dir, filename)] = labels.index(label)
        
# Get the filenames and labels
filenames = []
labels = []

# Loop through each folder in the directory
for class_name in os.listdir(image_dir):
    class_dir = os.path.join(image_dir, class_name)
    for filename in os.listdir(class_dir):
        filepath = os.path.join(class_dir, filename)
        filenames.append(filepath)
        labels.append(class_name)
        
# Create a train-validation split
train_filenames, test_filenames, train_labels, test_labels = train_test_split(
    filenames, labels, test_size=0.2, random_state=42)

# Define data directories
train_dir = 'C:/Users/maria/Documents/GitHub/CSCU9M6-2929300/Dataset/train'
test_dir = 'C:/Users/maria/Documents/GitHub/CSCU9M6-2929300/Dataset/test'
val_dir = 'C:/Users/maria/Documents/GitHub/CSCU9M6-2929300/Dataset/val'

# Create TensorFlow dataset
dataset = create_dataset(filenames, labels, batch_size)

# Print the shape of the dataset
print(dataset.element_spec)

(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.string, name=None))


---

# 5. **Proposed solution** 

This is where you should implement most of your code for your solution. Write the routines for training and predicting the models and any necessary intermediate steps. Post-processing functions must also be implemented here.

  - Use good programming practices, modularizing and adequately commenting on your code. Code quality will be considered in the final assessment.

  - You can use pre-trained models as backbones or any code available on the web as a basis, but they must be correctly credited and referenced both in this notebook and in the final report. Cite the source link repository and explicitly cite the authors of it.
If you changed existing code, make it clear what the changes were.
Make it clear where your own code starts and where it ends. Note that the originality percentage of the code will be considered in the evaluation, so use external codes wisely and sparingly. **Missconduct alert:** remember that there are many tools that compare existing source code and that it is relatively easy to identify authorship. So, be careful and fair by always properly thanking the authors if you use external code.

[top](#scrollTo=4i5afvUbhmGo)

In [5]:
img_size = (224, 224)

# Load and preprocess the training and testing datasets
train_dataset = create_dataset(train_filenames, train_labels, batch_size)
test_dataset = create_dataset(test_filenames, test_labels, batch_size)

# Define image data generator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(244, 244),
    batch_size=batch_size,
    class_mode='categorical')

# create model using VGG16 architecture
def create_model():
    vgg16_model = tf.keras.applications.VGG16(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
    for layer in vgg16_model.layers:
        layer.trainable = False
        
    model = keras.models.Sequential([
        keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(img_size[0]*2, img_size[1]*2, 3)),
        keras.layers.Flatten(),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(15, activation='softmax')])
    return model

# Define the model
input_shape = img_size + (3,)
model = keras.models.Sequential([
    keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=input_shape),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(15, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=val_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)

Found 128 images belonging to 15 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Epoch 1/10
4/4 [==============================] - 40s 10s/step - loss: 3.3874 - accuracy: 0.0859
Epoch 2/10
4/4 [==============================] - 64s 18s/step - loss: 3.6642 - accuracy: 0.1172
Epoch 3/10
4/4 [==============================] - 70s 17s/step - loss: 2.8792 - accuracy: 0.0625
Epoch 4/10
4/4 [==============================] - 74s 18s/step - loss: 2.7402 - accuracy: 0.0781
Epoch 5/10
4/4 [==============================] - 70s 17s/step - loss: 2.7510 - accuracy: 0.1172
Epoch 6/10
4/4 [==============================] - 69s 17s/step - loss: 2.7023 - accuracy: 0.0781
Epoch 7/10
4/4 [==============================] - 70s 17s/step - loss: 2.7040 - accuracy: 0.1172
Epoch 8/10
4/4 [==============================] - 70s 17s/step - loss: 2.6955 - accuracy: 0.1094
Epoch 9/10
4/4 [==============================] - 71s 18s/step - loss: 2.6998 - accuracy: 0.1094
Epoch 1

ValueError: Asked to retrieve element 0, but the Sequence has length 0

---

# 6. **Experimental tests and evaluations** 


Here you must implement your code for training, testing and evaluating your solution. For this, the following code blocks (*E1*, *E2*, and *E3*) are mandatory:

  - *E1* - Training the models. Implement code to call the dataloaders implemented for training your models.  Make routines to test different parameters of your models. Plot graphs that illustrate how parameters impact model training. Compare. Train and select a model for each city (A and B) and justify. You should use half (50%) of the samples from each dataset for training and leave the other half for testing (50%). 

[top](#scrollTo=4i5afvUbhmGo)

In [ ]:
# Write your codes for E1 here. Create more code cells if needed





  - *E2* - Testing the models in the dataset. You must implement code routines to test the predictive ability of your models using half of each dataset intended for testing. **The model trained in city A must be tested in city A. The model trained in city B must be tested in city B.** Use the evaluation metrics (accuracy, F1-score, AUC, etc) that are most appropriate for your problem. Plot graphs that illustrate the results obtained for each city (A and B). Plot visual examples of correctly (true positive) and incorrectly (false positive) classified samples. 

[top](#scrollTo=4i5afvUbhmGo)


In [ ]:
# Write your codes for E2 here. Create more code cells if needed





  - *E3* - Testing the models crossing datasets. Here you must do exactly the same as in *E2*, but now training in one city and testing in the other. **The model trained in city A must be tested in city B. The model trained in city B must be tested in city A.** Use the same metrics and plot the same types of graphs so that results are comparable.

[top](scrollTo=4i5afvUbhmGo)

In [ ]:
# Write your codes for E3 here. Create more code cells if needed





---

# 7. **Quiz and Report**

Answer the assessment quiz that will be made available on Canvas one week before the final deadline. Make a 2-page report using the [IEEE template](https://www.overleaf.com/read/rdqwshtvyjdn) with a maximum of 1000 words. Latex is recommended, but you can deliver the report in MS Word if you prefer. Your report should contain five sections: introduction, description of the proposed solution with justifications, results (here you can include the same graphs and pictures generated in this jupyter notebook), discussion of the results, and conclusion. Properly cite references to articles, tutorials, and sources used. A pdf version of your report should be made available in the project's github repository under the name "[project name] + _final_report.pdf".


[top](#scrollTo=4i5afvUbhmGo)